In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
cell_images = '../input/cell-images-for-detecting-malaria/cell_images/cell_images'

In [3]:
uninfected = len(os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected"))
parasitized = len(os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized"))
print(uninfected)
print(parasitized)

13780
13780


In [4]:
image_generator = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               shear_range=0.1,
                               fill_mode="constant")
# fill mode constant is the best idea since all the images have black background in the first place

In [5]:
train = tf.keras.preprocessing.image_dataset_from_directory(cell_images,
                                                            validation_split=0.2,
                                                            seed=123,
                                                            subset="training",
                                                            image_size=(155,155),
                                                            batch_size=32)
test = tf.keras.preprocessing.image_dataset_from_directory(cell_images,
                                                           validation_split=0.2,
                                                           seed=123,
                                                           subset="validation",
                                                           image_size=(155,155),
                                                           batch_size=32)



Found 27558 files belonging to 2 classes.
Using 22047 files for training.


2021-10-17 02:49:37.811227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-17 02:49:37.911265: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-17 02:49:37.911936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-17 02:49:37.915003: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [6]:
model = Sequential()

INPUT_SHAPE = (155,155,3)

model.add(Conv2D(32,(3,3),input_shape=INPUT_SHAPE,activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(64,(3,3),input_shape=INPUT_SHAPE,activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(128,(3,3),input_shape=INPUT_SHAPE,activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(256,(3,3),input_shape=INPUT_SHAPE,activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=["accuracy"])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 155, 155, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 77, 77, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 19, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 256)       2

In [8]:
early_stopping = EarlyStopping(monitor="val_loss",patience=3, verbose=True)

In [9]:
history = model.fit(train,validation_data=test,epochs=20,callbacks=[early_stopping],verbose=True)

Epoch 1/20


2021-10-17 02:49:42.559373: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-17 02:49:44.155222: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


689/689 [==============================] - 97s 129ms/step - loss: 0.9682 - accuracy: 0.5579 - val_loss: 0.6616 - val_accuracy: 0.6258
Epoch 2/20
689/689 [==============================] - 31s 45ms/step - loss: 0.6794 - accuracy: 0.5975 - val_loss: 0.6593 - val_accuracy: 0.6130
Epoch 3/20
689/689 [==============================] - 32s 45ms/step - loss: 0.6529 - accuracy: 0.6258 - val_loss: 0.6355 - val_accuracy: 0.6492
Epoch 4/20
689/689 [==============================] - 32s 46ms/step - loss: 0.6397 - accuracy: 0.6401 - val_loss: 0.3947 - val_accuracy: 0.9131
Epoch 5/20
689/689 [==============================] - 32s 46ms/step - loss: 0.2135 - accuracy: 0.9388 - val_loss: 0.1531 - val_accuracy: 0.9501
Epoch 6/20
689/689 [==============================] - 32s 46ms/step - loss: 0.1673 - accuracy: 0.9495 - val_loss: 0.1319 - val_accuracy: 0.9555
Epoch 7/20
689/689 [==============================] - 32s 46ms/step - loss: 0.1513 - accuracy: 0.9549 - val_loss: 0.1334 - val_accuracy: 0.9572
Ep